In [ ]:
%pip install numpy opencv-python matplotlib torch torchvision numba

In [ ]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
import torch
from torchvision import models, transforms
from torchvision.models.segmentation import deeplabv3_mobilenet_v3_large
from numba import jit


In [ ]:
def load_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image

In [ ]:
def compute_energy_map(image):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    sobel_x = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
    sobel_y = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)
    energy_map = np.abs(sobel_x) + np.abs(sobel_y)
    return energy_map


In [ ]:
from torchvision.models.segmentation import deeplabv3_mobilenet_v3_large

def load_segmentation_model():
    model = deeplabv3_mobilenet_v3_large(pretrained=True).eval()
    preprocess = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    return model, preprocess


In [ ]:
def get_segmentation_mask(image, model, preprocess):
    input_tensor = preprocess(image).unsqueeze(0)
    with torch.no_grad():
        output = model(input_tensor)['out']
    segmentation_map = torch.argmax(output.squeeze(), dim=0).detach().cpu().numpy()
    return segmentation_map

In [ ]:
def enhance_energy_map(energy_map, segmentation_mask, important_classes, lambda_weight=1000):
    enhanced_map = energy_map.copy()
    for cls in important_classes:
        enhanced_map[segmentation_mask == cls] += lambda_weight
    return enhanced_map

In [ ]:
def find_seam(energy_map):
    rows, cols = energy_map.shape
    seam = np.zeros(rows, dtype=int)
    cost = energy_map.copy()
    backtrack = np.zeros_like(cost, dtype=int)

    for i in range(1, rows):
        for j in range(cols):
            left = cost[i - 1, j - 1] if j > 0 else float('inf')
            up = cost[i - 1, j]
            right = cost[i - 1, j + 1] if j < cols - 1 else float('inf')
            min_energy = min(left, up, right)

            cost[i, j] += min_energy
            backtrack[i, j] = np.argmin([left, up, right]) - 1

    seam[-1] = np.argmin(cost[-1])
    for i in range(rows - 2, -1, -1):
        seam[i] = seam[i + 1] + backtrack[i + 1, seam[i + 1]]

    return seam

In [ ]:

def remove_seam(image, seam):
    rows, cols, _ = image.shape
    output = np.zeros((rows, cols - 1, 3), dtype=image.dtype)
    for i in range(rows):
        output[i, :, :] = np.delete(image[i, :, :], seam[i], axis=0)
    return output


In [ ]:
def seam_carve(image, energy_map, num_seams=1):
    for _ in range(num_seams):
        seam = find_seam(energy_map)
        image = remove_seam(image, seam)
        energy_map = compute_energy_map(image)
    return image

In [ ]:
def main(image_path, important_classes=[15]):  # Example: Class 15 is "person"
    image = load_image(image_path)
    model, preprocess = load_segmentation_model()


    energy_map = compute_energy_map(image)
    segmentation_mask = get_segmentation_mask(image, model, preprocess)

    enhanced_map = enhance_energy_map(energy_map, segmentation_mask, important_classes)

    # Perform enhanced seam carving
    resized_image = seam_carve(image, enhanced_map, num_seams=100)
    resized_image_2 = seam_carve(image, energy_map, num_seams=100)

    cv2.imwrite("original_image.jpg", cv2.cvtColor(image, cv2.COLOR_RGB2BGR))
    cv2.imwrite("vanilla_resized_image.jpg", cv2.cvtColor(resized_image_2, cv2.COLOR_RGB2BGR))
    cv2.imwrite("segmentation_mask.jpg", (segmentation_mask * 255).astype(np.uint8))
    cv2.imwrite("enhanced_resized_image.jpg", cv2.cvtColor(resized_image, cv2.COLOR_RGB2BGR))


    # Show results
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 3, 1)
    plt.title("Original Image")
    plt.imshow(image)
    plt.axis("off")

    plt.subplot(1, 3, 2)
    plt.title("Segmentation Mask")
    plt.imshow(segmentation_mask, cmap="jet")
    plt.axis("off")


    plt.subplot(1, 3, 3)
    plt.title("Vanillla Resized Image")
    plt.imshow(resized_image_2)
    plt.axis("off")
    plt.show()

    plt.subplot(1, 3, 3)
    plt.title("Enhanced Resized Image")
    plt.imshow(resized_image)
    plt.axis("off")
    plt.show()


    # from google.colab import files

    # files.download("original_image.jpg")
    # files.download("segmentation_mask.jpg")
    # files.download("resized_image.jpg")


In [ ]:
if __name__ == "__main__":
  image_path = "place_image_here"
  main(image_path)
